In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras import Sequential,Model
from keras.layers import Conv1D,Input,Softmax,concatenate,InputLayer,Flatten,Dense,Activation,MaxPool1D
import tensorflow as tf

In [46]:
df = pd.read_csv('D:\\Career\\GradProj\\Datasets\\dataset3.csv')

In [ ]:
# df_windows = pd.DataFrame(columns=list(df.columns))
# df_r_windows = pd.DataFrame(columns=list(df.columns))
# np.random.seed(42)
# for i in range(0,len(df),25):
#     df_window = df.loc[i:i+75]
#     df_random = df.loc[i:i+75]
#     df_window['window number'] = int(i)
#     df_random['window number'] = int(i)
#     df_random = np.random.shuffle(np.asarray(df_random).astype(np.float32))
    
#     df_windows = pd.concat([df_windows, df_window], ignore_index=True)
#     df_r_windows = pd.concat([df_r_windows, df_random], ignore_index=True)
#     print(f'\r{i}         ',end = '')
# df_r_windows.drop(['Unnamed: 0','Unnamed: 0.1'],axis = 1, inplace = True)

In [47]:
df_windows = pd.DataFrame(columns=list(df.columns))
for i in range(0,len(df),25):
    df_window = df.loc[i:i+75]
    df_random = df_window.copy()
    np.random.shuffle(np.asarray(df_random))
    df_windows = pd.concat([df_windows, df_window, df_random], ignore_index=True)

In [49]:
df_windows.drop(['Unnamed: 0'],axis = 1, inplace = True)

In [50]:
df_windows.shape

(344228, 61)

In [51]:
# df_windows.to_csv('D:\\Career\\GradProj\\Datasets\\dataset5_noisy.csv')
df_windows.to_csv('D:\\Career\\GradProj\\Datasets\\dataset4_random.csv')

In [ ]:
df_windows = pd.read_csv('D:\\Career\\GradProj\\Datasets\\dataset4_random.csv')

In [52]:
df_windows.columns

Index(['Unnamed: 0.1', '7 x', '8 x', '9 x', '10 x', '11 x', '12 x', '7 y',
       '8 y', '9 y', '10 y', '11 y', '12 y', '7 z', '8 z', '9 z', '10 z',
       '11 z', '12 z', '7 x d', '8 x d', '9 x d', '10 x d', '11 x d', '12 x d',
       '7 y d', '8 y d', '9 y d', '10 y d', '11 y d', '12 y d', '7 z d',
       '8 z d', '9 z d', '10 z d', '11 z d', '12 z d', '7 x dd', '8 x dd',
       '9 x dd', '10 x dd', '11 x dd', '12 x dd', '7 y dd', '8 y dd', '9 y dd',
       '10 y dd', '11 y dd', '12 y dd', '7 z dd', '8 z dd', '9 z dd',
       '10 z dd', '11 z dd', '12 z dd', 'LKA', 'LEA', 'REA', 'RKA', 'LHA',
       'RHA'],
      dtype='object')

In [53]:
df_windows.drop(['Unnamed: 0.1'],axis = 1, inplace = True)

In [54]:
df_windows_array = np.array(df_windows.iloc[:342000]).reshape(-1,75,60)
# df_r_windows_array = np.array(df_r_windows.iloc[:171000]).reshape(-1,75,60)
# df_total = np.concatenate((df_windows_array, df_r_windows_array), axis=0)

In [55]:
df_windows_array.shape

(4560, 75, 60)

In [56]:
target = np.tile([0,1],4560//2)
target

array([0, 1, 0, ..., 1, 0, 1])

In [104]:
# Adding callbacks

# class CustomSaver(tf.keras.callbacks.Callback):
#     def on_epoch_end(self,model , epoch, logs={}):
#         if epoch == 5:  # or save after some epoch, each k-th epoch etc.
#             self.model.save(f"D:\\Career\\GradProj\\Model\\All_models\\{epoch}_conv1d.hd5")

# saver = CustomSaver()
rlr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="loss",
    factor=0.6,
    patience=3,
    )

checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='D:\\Career\\GradProj\\Model\\Best_model_1_conv1d.hdf5', verbose=0, save_best_only=True)

csv_logger = tf.keras.callbacks.CSVLogger("D:\\Career\\GradProj\\Model\\model_history_log_conv1d.csv", append=True)

In [59]:
from keras.layers import Dropout,MaxPooling1D
from sklearn.model_selection import train_test_split

In [70]:
def cnn(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    # head 1
    inputs1 = Input(shape=(n_timesteps,n_features))
    conv1 = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs1)
    drop1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling1D(pool_size=2)(drop1)
    flat1 = Flatten()(pool1)
    # head 2
    inputs2 = Input(shape=(n_timesteps,n_features))
    conv2 = Conv1D(filters=64, kernel_size=5, activation='relu')(inputs2)
    drop2 = Dropout(0.5)(conv2)
    pool2 = MaxPooling1D(pool_size=2)(drop2)
    flat2 = Flatten()(pool2)
    # head 3
    inputs3 = Input(shape=(n_timesteps,n_features))
    conv3 = Conv1D(filters=64, kernel_size=11, activation='relu')(inputs3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling1D(pool_size=2)(drop3)
    flat3 = Flatten()(pool3)
    # merge
    merged = concatenate([flat1, flat2, flat3])
    # interpretation
    dense1 = Dense(100, activation='relu')(merged)
    outputs = Dense(1, activation='softmax')(dense1)
    model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [100]:
from keras.layers import LSTM,RepeatVector

In [101]:
def lstm():
    model = Sequential()
    model.add(Input((75, 60)))
    model.add(LSTM(32))
    model.add(RepeatVector(75))
    model.add(LSTM(16))
    model.add(Dense(16))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [60]:
X_train, X_test, y_train, y_test = train_test_split(df_windows_array, target, test_size=0.20, shuffle=False)

In [74]:
model = cnn(X_train,y_train,X_test,y_test)

In [102]:
model2 = lstm()

In [75]:
df_windows_array = np.asarray(df_windows_array).astype(np.float32)
target = np.asarray(target).astype(np.float32)

In [76]:
df_windows_array.shape

(4560, 75, 60)

In [77]:
# training
model.fit(  [X_train,X_train,X_train],
            y_train,
            epochs=100,
            callbacks = [rlr,checkpointer,csv_logger],
            batch_size=12)

Epoch 1/100
304/304 [==============================] - 4s 11ms/step - loss: 0.0000e+00 - accuracy: 0.5000 - lr: 0.0010
Epoch 2/100
304/304 [==============================] - 3s 11ms/step - loss: 0.0000e+00 - accuracy: 0.5000 - lr: 0.0010
Epoch 3/100
304/304 [==============================] - 3s 11ms/step - loss: 0.0000e+00 - accuracy: 0.5000 - lr: 0.0010
Epoch 4/100
304/304 [==============================] - 3s 11ms/step - loss: 0.0000e+00 - accuracy: 0.5000 - lr: 0.0010
Epoch 5/100
304/304 [==============================] - 3s 11ms/step - loss: 0.0000e+00 - accuracy: 0.5000 - lr: 0.0010
Epoch 6/100
304/304 [==============================] - 3s 10ms/step - loss: 0.0000e+00 - accuracy: 0.5000 - lr: 0.0010
Epoch 7/100
161/304 [==============>...............] - ETA: 1s - loss: 0.0000e+00 - accuracy: 0.4876

KeyboardInterrupt: 

In [105]:
# training
model2.fit(  X_train,
            y_train,
            epochs=100,
            callbacks = [rlr,checkpointer,csv_logger],
            batch_size=32)

Epoch 1/100
114/114 [==============================] - 2s 20ms/step - loss: 0.2495 - lr: 0.0010
Epoch 2/100
114/114 [==============================] - 2s 19ms/step - loss: 0.2499 - lr: 0.0010
Epoch 3/100
114/114 [==============================] - 2s 19ms/step - loss: 0.2485 - lr: 0.0010
Epoch 4/100
114/114 [==============================] - 2s 19ms/step - loss: 0.2491 - lr: 0.0010
Epoch 5/100
114/114 [==============================] - 2s 19ms/step - loss: 0.2483 - lr: 0.0010
Epoch 6/100
114/114 [==============================] - 2s 19ms/step - loss: 0.2486 - lr: 0.0010
Epoch 7/100
114/114 [==============================] - 2s 19ms/step - loss: 0.2488 - lr: 0.0010
Epoch 8/100
114/114 [==============================] - 2s 19ms/step - loss: 0.2489 - lr: 0.0010
Epoch 9/100
114/114 [==============================] - 2s 19ms/step - loss: 0.2477 - lr: 6.0000e-04
Epoch 10/100
114/114 [==============================] - 2s 19ms/step - loss: 0.2467 - lr: 6.0000e-04
Epoch 11/100
114/114 [=========

KeyboardInterrupt: 

In [ ]:
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

(None, 75, 60) <dtype: 'float32'>
(None, 1) <dtype: 'float32'>
conv1d (None, 75, 60) float32
conv1d_1 (None, 66, 50) float32
conv1d_2 (None, 57, 40) float32
flatten (None, 48, 30) float32
dense (None, 1440) float32
dense_1 (None, 10) float32
activation (None, 1) float32


[None, None, None, None, None, None, None]